In [34]:
import tensorflow as tf
import tensorflow_datasets as tfds
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import os


In [35]:
import sys
sys.path.append(r"C:\DIT826\team1") 
print(sys.path)

import os
os.getcwd()
import sentio
import sentio.settings.development
os.environ.setdefault(
    "DJANGO_SETTINGS_MODULE",
    "sentio.settings.development"  
)

import django

try:
    django.setup()
except Exception as e:
    import traceback
    traceback.print_exc()

['C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\python310.zip', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\DLLs', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\lib', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab', '', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\lib\\site-packages', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\lib\\site-packages\\win32', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Waggi\\.conda\\envs\\cnnlab\\lib\\site-packages\\setuptools\\_vendor', 'C:\\DIT826\\team1', 'C:\\DIT826\\team1', 'C:\\DIT826\\team1']


In [36]:
from apps.ml_admin.models import DatasetRecord
from asgiref.sync import sync_to_async
# Load data training data into a pandas dataframe


@sync_to_async
def extract_data_async():

    qs = DatasetRecord.objects.all()
    print(qs.exists())  # True if there are any records
    print(qs.count())   # Number of records

    qs = DatasetRecord.objects.all().values(
        "text",
        "label",
        "category_id"
    )

    #df = pd.DataFrame(list(qs))
    df = pd.DataFrame(list(DatasetRecord.objects.all().values()))
    df.columns

    # Ensure numeric category aligns with the text label
    label_map = {
        "normal": 0,
        "depression": 1,
        "suicidal": 2,
        "stress": 3,
    }
    #df["status"] = df["label"].map(label_map)

    return df




df = await extract_data_async()

df.head()




True
50590


,id,text,label,category_id,normal,depression,suicidal,stress,dataset_type,data_upload_id,imported_at
0,1,oh my gosh,Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.944463+00:00
1,2,"trouble sleeping, confused mind, restless hear...",Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.944463+00:00
2,3,"All wrong, back off dear, forward doubt. Stay ...",Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.944463+00:00
3,4,I've shifted my focus to something else but I'...,Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.945475+00:00
4,5,"I'm restless and restless, it's been a month n...",Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.945475+00:00


In [37]:
# Clean and normalize labels
df['label'] = df['label'].astype(str).str.strip().str.lower()

label_map = {
    "normal": 0,
    "depression": 1,
    "suicidal": 2,
    "stress": 3,
}

# Map labels
df['label'] = df['label'].map(label_map)

# Drop any unmapped labels
df = df.dropna(subset=['label'])

# Convert to int
df['label'] = df['label'].astype(int)


In [38]:


train_df = df.sample(frac=0.75, random_state=42)
test_df  = df.drop(train_df.index)

train_ds = tf.data.Dataset.from_tensor_slices((
    train_df['text'].values,
    train_df['label'].values
))

test_ds = tf.data.Dataset.from_tensor_slices((
    test_df['text'].values,
    test_df['label'].values
))

train_ds = train_ds.shuffle(10000).batch(32)
test_ds = test_ds.batch(32)

In [39]:
# Create encoder (vectorizer)
encoder = tf.keras.layers.TextVectorization(
    max_tokens=10000,
    output_sequence_length=200
)

# Adapt using *unbatched* text data
encoder.adapt(
    train_ds.unbatch().map(lambda text, label: text)
)

# Get vocabulary
vocabulary = np.array(encoder.get_vocabulary())

# Take a single example from dataset
example_text, example_label = next(iter(train_ds.unbatch()))

# Encode
encoded = encoder(example_text)
encoded_numpy = encoded.numpy()

# Decode back to words
decoded = ' '.join(vocabulary[encoded_numpy])

print('Original:', example_text.numpy().decode())
print('Encoded:', encoded_numpy)
print('Decoded:', decoded)


Original: Random itchiness? Just looking for possibilities I guess, here's what happened: I'm preparing to move by putting stuff into boxes, today I focused on decorations such as figurines around the house, military memorabilia, books, and small miscellaneous items. The only thing out of the ordinary was an old roommate's uniform that moved out like 6 months ago that was sitting under where I put my motorcycle helmet, I picked it up and unfurled it, saw it was his put it in my roommates pile of stuff. About 45 minutes later (give or take) my arms became itchy, incredibly itchy. When I looked at it it looked like a rash all on one side of my forearm (basically exactly where the uniform touched) and then a couple of welt like spots a bit away. I washed my arms but a little bit after I was itchy on the middle of my back, shoulder, upper thighs, calves, ankles, chest, and stomach. I hopped immediately in the shower and changed clothes. There is a chance it was old fuel on the uniform, I h

In [40]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1,), dtype=tf.string),
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(7)  # 7 classes
])


In [41]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)


In [42]:
labels = train_df['label'].values
#Automatically creates class weights based on how common the class is in the dataset
print(np.unique(labels))
print(labels[:20])
class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(labels), y=labels
)
class_weights_dict = dict(enumerate(class_weights))


history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    class_weight=class_weights_dict
)


[0 1 2 3]
[1 3 2 0 0 2 0 0 0 1 1 2 2 0 1 2 1 0 2 1]
Epoch 1/10
1186/1186 [==============================] - 540s 444ms/step - loss: 0.7718 - accuracy: 0.6444 - val_loss: 0.6062 - val_accuracy: 0.7064
Epoch 2/10
1186/1186 [==============================] - 1233s 1s/step - loss: 0.5250 - accuracy: 0.7565 - val_loss: 0.5453 - val_accuracy: 0.7626
Epoch 3/10
1186/1186 [==============================] - 630s 531ms/step - loss: 0.4397 - accuracy: 0.8068 - val_loss: 0.5284 - val_accuracy: 0.7746
Epoch 4/10
1186/1186 [==============================] - 632s 533ms/step - loss: 0.4176 - accuracy: 0.8225 - val_loss: 0.5488 - val_accuracy: 0.7757
Epoch 5/10
1186/1186 [==============================] - 1589s 1s/step - loss: 0.3453 - accuracy: 0.8540 - val_loss: 0.5279 - val_accuracy: 0.7926
Epoch 6/10
1186/1186 [==============================] - 964s 813ms/step - loss: 0.3016 - accuracy: 0.8736 - val_loss: 0.5620 - val_accuracy: 0.7893
Epoch 7/10
1186/1186 [==============================] - 673s 568

In [43]:
loss, accuracy = model.evaluate(test_ds)
print(f"Test accuracy: {accuracy:.4f}")


396/396 [==============================] - 55s 138ms/step - loss: 0.7232 - accuracy: 0.7769
Test accuracy: 0.7769


In [45]:
import joblib
save_path = "ml_pipeline/toy_models/RNNLSTM.pkl"
joblib.dump(model, "RNNLSTM.pkl")


WARNING Found untraced functions such as lstm_cell_13_layer_call_fn, lstm_cell_13_layer_call_and_return_conditional_losses, lstm_cell_14_layer_call_fn, lstm_cell_14_layer_call_and_return_conditional_losses, lstm_cell_16_layer_call_fn while saving (showing 5 of 8). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: ram://56a76248-796b-43db-b368-87f2bc1c376c/assets


INFO Assets written to: ram://56a76248-796b-43db-b368-87f2bc1c376c/assets


['RNNLSTM.pkl']

In [53]:
y_true = []
y_pred = []

for x, y in test_ds:
    preds = model.predict(x, verbose=0)
    y_true.extend(y.numpy())
    y_pred.extend(np.argmax(preds, axis=1))


In [50]:
from sklearn.metrics import classification_report, accuracy_score

metrics = classification_report(y_true, y_pred, target_names=["normal", "depression", "suicidal", "stress"], output_dict=True)
accuracy = accuracy_score(y_true, y_pred)


In [52]:
from apps.ml_admin.models import ModelVersion
from django.contrib.auth.models import User
save_path = "ml_pipeline/toy_models/RNNLSTM.pkl"

@sync_to_async
def get_user():
    user = User.objects.first()

@sync_to_async
def create_model_version():
    ModelVersion.objects.create(
    version_name="v1.1",
    model_file_path=save_path,
    accuracy=accuracy,
    precision=float(metrics["depression"]["precision"]),
    recall=float(metrics["depression"]["recall"]),
    f1_score=float(metrics["depression"]["f1-score"]),
    is_active=True,
    created_by=user
    )

user = await get_user()
print(user)
await create_model_version()

None


IntegrityError: UNIQUE constraint failed: ml_admin_modelversion.version_name

In [122]:
new_texts = [
    "I feel very stressed and anxious these days.",
    "I am happy and I feel normal"
]

predictions = model.predict(new_texts)

predicted_classes = tf.argmax(predictions, axis=1).numpy()

print(predicted_classes)

inv_label_map = {v: k for k, v in label_map.items()}

for i, text in enumerate(new_texts):
    print(f"Text: {text}")
    print(f"Predicted label: {inv_label_map[predicted_classes[i]]}")


1/1 [==============================] - 12s 12s/step
[4 1]
Text: I feel very stressed and anxious these days.
Predicted label: Stress
Text: I am happy and I feel normal
Predicted label: Depression


In [124]:
preds = model.predict(["Everything is on fire around me and I don't know what to do"])
probs = tf.nn.softmax(preds, axis=1).numpy()
for i, prob in enumerate(probs[0]):
    print(f"{inv_label_map[i]}: {prob:.2f}")


1/1 [==============================] - 0s 226ms/step
Normal: 0.65
Depression: 0.22
Suicidal: 0.12
Anxiety: 0.01
Stress: 0.00
Bipolar: 0.00
Personality disorder: 0.00
